# 부산광역시 인구동태 현황으로 스타벅스 지점과의 연관성 찾기

In [1]:
# !pip install matplotlib
# !pip install pandas

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.dates as mdates
import pandas as pd
import numpy as np

In [3]:
# !pwd # !dir/w

'pwd'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [4]:
file_path = '../ref/부산광역시 주민등록인구통계_20231231.csv'
# CSV 파일읽기
ingu_data = pd.read_csv(file_path, encoding='EUC-KR')
ingu_data = ingu_data[['구군','인구수(명)']]
ingu_data = ingu_data.sort_values(by='인구수(명)', ascending= False )
ingu_data.head()

FileNotFoundError: [Errno 2] No such file or directory: '../ref/부산광역시 주민등록인구통계_20231231.csv'

In [ ]:
star_path = '../ref/starbucks_20250411113937.csv'
star_data = pd.read_csv(star_path, encoding='utf-8')
star_data.columns = ['no.','name','lat','lon','addr','type']
star_data

In [ ]:
star_data.sample(10)

In [ ]:
# 방법1
# gu = []
# addr_list = star_data['addr']
# for n in addr_list:
#     gu.append(n.split(" ")[1])
# star_data['구'] = gu
# star_data
# 방법2
star_data['구군'] = [ a.split(" ")[1] for a in star_data['addr']]
star_data

In [ ]:
# 리눅스용 폰트 경로 설정
fontlist = fm.findSystemFonts(fontpaths=None, fontext='ttf')
fontlist
plt.rc('font', family="NanumBarunGothic")
plt.text(0.2,0.4,'한글',size=100)

In [ ]:
star_sum = star_data.groupby('구군')['구군'].count().reset_index(name="스벅수")
star_sum.plot(kind='bar', x='구군', y="스벅수")
plt.xticks(rotation=45)
plt.show()

In [ ]:
star_sum

In [ ]:
total = pd.merge(ingu_data, star_sum , on='구군', how = 'left')     
total = total.rename(columns={"인구수(명)":"인구수"})
total

부산 산업대분류, 읍·면·동별 사업체수, 종사자수 <br>
https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=INH_1K52C01_21&conn_path=I2

In [ ]:
biz_path = "../ref/부산광역시구분별사업체수종사자수_2020.csv"
biz_data = pd.read_csv(biz_path, encoding="EUC-KR")
biz_data.columns = ['year','구군','type1','사업체수','종사자수']
biz = biz_data[['구군','사업체수','종사자수']]
biz

In [ ]:
total_m1 = pd.merge(total, biz, on='구군', how='left')
total_m1

### 주소를 가지고 위도 / 경도 값 얻어오기(GPS 정보 얻기) 1

In [ ]:
# GPS정보로 지도에 표시하기
gps_path ="../ref/gps_test.csv"
gps = pd.read_csv(gps_path,encoding="utf-8")
gps

In [ ]:
# !pip install folium

In [ ]:
import folium
coordinates = gps[['Latitude','Longitude']].to_numpy()
place = gps['주소'].to_numpy()

# 중심좌표 설정(기본위치)
center_lat , center_lon = 35.17574571 , 129.0817409

# 지도생성
m = folium.Map(location=[center_lat , center_lon], zoom_start=12 )


for i, (lat, lon) in enumerate(coordinates):
    folium.CircleMarker(
        location=[lat , lon],
        radius = 30,
        color= 'orange',
        weight=1,  # 테두리 굵기 (기본값: 2)
        opacity=0.5,  # 테두리 투명도 (0.0 ~ 1.0)
        fill = True,
        fill_color = 'yellow',
        fill_opacity = 0.6,
        tooltip = place[i]
    ).add_to(m) # 지도 생성 후 원을 그리고 지도에 붙임

    folium.Marker(
        location = [lat + 0.001, lon],
        icon = folium.DivIcon(f'<div style="font-size: 9px; color: black; background-color: white; width:50px; white-space: nowrap; overflow: hidden; text-overflow: ellipsis; opactiy:0.7; border-radius:5px">{place[i]}</div>')
    ).add_to(m)
    
m

### 주소를 가지고 위도 / 경도 값 얻어오기(GPS 정보 얻기) 2

In [ ]:
import geopandas as gpd
import pandas as pd
gdf = gpd.read_file("../ref/LARD_ADM_SECT_SGG_26_202502.shp", encoding='cp949')

# 현재 좌표계 확인
print(f"현재 좌표계: {gdf.crs}")

# WSG84 좌표계로 변환 (EPSG:4326)
gdf = gdf.to_crs(epsg=4326)
coordinates=[]
for idx, row in gdf.iterrows():
    latitude = row.geometry.centroid.y
    longitude = row.geometry.centroid.x
    coordinates.append([row['SGG_NM'],latitude, longitude])
    print(f"{row['SGG_NM']}의 중심점 좌표: 위도 {latitude}, 경도 {longitude}")

geo = pd.DataFrame(coordinates)
geo.columns=['구군','위도','경도']
geo['구군']=geo['구군'].str.replace('부산광역시 ','')
geo

### 주소를 가지고 위도 / 경도 값 얻어오기(GPS 정보 얻기) 3

In [ ]:
# !pip install geopy

In [ ]:
from geopy.geocoders import Nominatim

loc = "부산광역시 남구 대연동 1673"
geolocator = Nominatim(user_agent="geoapi")
location = geolocator.geocode(loc)
print(location.latitude, location.longitude)

In [ ]:
last= pd.merge(total_m1, geo, on="구군", how="left")
last.info()
last.sample(5)

In [ ]:
import geopandas as gpd
import json

In [ ]:
geojson = gdf.to_json()

In [ ]:
import folium
location_mean = last[["위도","경도"]].mean()
print(location_mean)

In [ ]:
def style_func(feature):
    return {'opacity':0.7, 'weight':1.5, 'color':'red', 'fillOpacity':0, 'dashArray':'5,5'}
    # opacity 투명도를 설정
    # weight 외곽선의 두께를 설정
    # color 경계선의 색상
    # fillOpacity 면적을 채우는 색상의 투명도를 설정
    # dashArray 'dashArray':'5,5'는 5px 길이의 선과 5px 길이의 간격을 반복

sb_bubble = folium.Map(location= list(location_mean), tiles= 'CartoDB Positron', zoom_start=12) # Positron, Voyager, Dark Matter
folium.GeoJson(geojson, style_function=style_func).add_to(sb_bubble)

# 지도 출력 확인
sb_bubble

In [ ]:
sb_mean = last['스벅수'].mean()
sb_mean

In [ ]:
def style_func(feature):
    return {'opacity':0.7, 'weight':1.5, 'color':'red', 'fillOpacity':0, 'dashArray':'5,5'}
sb_bubble = folium.Map(location= list(location_mean), tiles= 'CartoDB Positron', zoom_start=12) # Positron, Voyager, Dark Matter
folium.GeoJson(geojson, style_function=style_func).add_to(sb_bubble)

for i in last.index:    
    lat = float(last.loc[i, '위도'])
    lon = float(last.loc[i, '경도'])
    sb = int(last.loc[i,'스벅수'])

    if sb > sb_mean:
        fillColor = 'red'
    else:
        fillColor = 'blue'

    folium.CircleMarker(
        location=[lat,lon],
        color='yellow',
        fill_color = fillColor,
        fill_opacity=0.7,
        weight=2,
        radius=(sb*2),
     ).add_to(sb_bubble)

# 지도 출력 확인
sb_bubble

In [ ]:
# 단계 구분도 시각화
# coordinates
import json
print(type(geojson))
geojson_data = json.loads(geojson)
print(type(geojson_data))
for n in range(len(geojson_data['features'])):
    print(geojson_data['features'][0])

# DIC구조로 접근 X -> Geojson 계층적 구조

In [ ]:
threshold_scale = [last['스벅수'].min(), last['스벅수'].quantile(0.25), last['스벅수'].quantile(0.5), last['스벅수'].quantile(0.75), last['스벅수'].max()]
sb_bubble2 = folium.Map(location=list(location_mean), tiles='CartoDB Voyager', zoom_start=11)

# 단계구분도시각화
folium.Choropleth(
    geo_data=geojson_data,
    data = last,
    columns=['구군','스벅수'],
    fill_color='PuBuGn',
    fill_opacity = 0.7,
    line_opacity=0.5,
    key_on ='feature.properties.SGG_NM', # GEOJSON 형식으로 접근 방식.
    threshold_scale = threshold_scale,
    legend_name = "스타벅스 수"
).add_to(sb_bubble2)
sb_bubble2
plt.show()
plt.savefig('./구군스벅수.png')

In [ ]:
threshold_scale = [last['인구수'].min(), last['인구수'].quantile(0.25), last['인구수'].quantile(0.5), last['인구수'].quantile(0.75), last['인구수'].max()]
sb_bubble3 = folium.Map(location=list(location_mean), tiles='CartoDB Voyager', zoom_start=10)

# last['인구수']= last['인구수'].astype(int) 스트링 값을 숫자값으로 변환
# 단계구분도시각화
folium.Choropleth(
    geo_data=geojson_data,
    data = last,
    columns=['구군','인구수'],
    fill_color='YlOrBr',
    fill_opacity = 0.7,
    line_opacity=0.5,
    key_on ='feature.properties.SGG_NM', # GEOJSON 형식으로 접근 방식.
    threshold_scale = threshold_scale,
    legend_name = "인구 수"
).add_to(sb_bubble3)
sb_bubble3
plt.savefig('./구군인구수.png')

In [ ]:
threshold_scale = [last['사업체수'].min(), last['사업체수'].quantile(0.25), last['사업체수'].quantile(0.5), last['사업체수'].quantile(0.75), last['사업체수'].max()]
sb_bubble4 = folium.Map(location=list(location_mean), tiles='CartoDB Voyager', zoom_start=11)

# last['사업체수']= last['사업체수'].astype(int) 스트링 값을 숫자값으로 변환
# 단계구분도시각화
folium.Choropleth(
    geo_data=geojson_data,
    data = last,
    columns=['구군','사업체수'],
    fill_color='YlGnBu',
    fill_opacity = 0.7,
    line_opacity=0.5,
    key_on ='feature.properties.SGG_NM', # GEOJSON 형식으로 접근 방식.
    threshold_scale = threshold_scale,
    legend_name = "인구 수"
).add_to(sb_bubble4)
sb_bubble4
plt.savefig('./구군사업체수.png')